In [1]:
from matplotlib import pyplot as plt
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix

**Reading Data and Checking for NULL Values**

In [4]:
for i in range(1, 16):
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Major-project-8th-sem/data_num/ds'+ str(i) +'.csv')
    print('='*50)
    print('shape of {}:'.format(i), df.shape)
    print(df.columns)
    if df.isnull().sum().sum() != 0:
        print(df.isnull().sum())
        break

shape of 1: (250, 13)
Index(['surgery', 'Age', 'Hospital_Number', 'pulse', 'mucous_membranes',
       'capillary_refill_time', 'peristalsis', 'packed_cell_volume',
       'total_protein', 'outcome', 'surgical_lesion', 'site_of_lesion',
       'pathology_cp_data'],
      dtype='object')
shape of 2: (250, 10)
Index(['surgery', 'Age', 'pulse', 'mucous_membranes', 'capillary_refill_time',
       'peristalsis', 'packed_cell_volume', 'total_protein', 'outcome',
       'surgical_lesion'],
      dtype='object')
shape of 3: (306, 4)
Index(['Age_of_patient_at_time_of_operation', 'Patients_year_of_operation',
       'Number_of_positive_axillary_nodes_detected', 'Survival_status'],
      dtype='object')
shape of 4: (310, 7)
Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'Class'], dtype='object')
shape of 5: (320, 7)
Index(['ACTION', 'CONDITIONAL', 'CONTINUANCE', 'IMPERATIVE', 'RISK_LEVEL',
       'SOURCE', 'DEFECT'],
      dtype='object')
shape of 6: (327, 38)
Index(['a', 'b', 'c', 'd', 'e', 'f', 'g',

**EXPERIMENT DEFINITION**

In [6]:
from sklearn.linear_model import Perceptron

def linear(X,theta):
    z = np.dot(X,theta.T)
    return z

def cost_ada(z,y):
    loss = ((y-z)**2).sum()
    return loss/2.0

def gd_ada(X,z,y):
    return -np.dot((y-z), X)

def cost_ada2(z,y):              # MSE
    loss = ((y-z)**2).sum()
    return loss/len(y)

def gd_ada2(X,z,y):               # MSE
    return -np.dot((y-z), X)/len(y)


def predict_p(X,theta):
    outcome = []
    result = linear(X,theta)
    for i in range(X.shape[0]):
        if result[i] <= 0:
            outcome.append(-1)
        else:
            outcome.append(1)
    return outcome

def sigmoid(X,theta):
    z = np.dot(X,theta.T).astype(float)
    return 1.0/(1+np.exp(-z))

def cost_function(h,y):
    loss = ((-y * np.log(h))-((1-y)* np.log(1-h))).mean()
    return loss

def gradient_descent(X,h,y,yl):
    return np.dot((h-y), X)/yl

def predict(X,theta):
    outcome = []
    result = sigmoid(X,theta)
    for i in range(X.shape[0]):
        if result[i] <= threshold:
            outcome.append(0)
        else:
            outcome.append(1)
    return outcome


def predict_ours(X,theta):
    outcome = []
    result = sigmoid_2(sigmoid(X,theta))
    for i in range(X.shape[0]):
        if result[i] <= threshold:
            outcome.append(0)
        else:
            outcome.append(1)
    return outcome

def sigmoid_2(p):
    s = (L*(p-0.5)).astype(float)
    return 1.0/(1+np.exp(-s))

def cost_function_fscore(bs,syh,syhy,bs_sy):
    loss = (1+bs)*syhy / ( bs_sy + syh )
    return 1-loss

def gradient_descent_fscore(p,X,yh,y,bs,syh,syhy,bs_sy):
    yp_pz = L*yh*(1-yh) * p*(1-p)
    return -( (1+bs) * (np.dot(y*yp_pz*(bs_sy+syh), X) - np.dot(yp_pz*syhy, X)) ) / ( (bs_sy + syh)**2 )


def cost_function_acc(sy,syh,syhy,yl):
    loss = (yl-sy-syh+2*syhy)/yl   # accuracy
    return 1-loss

def gradient_descent_acc(p,X,yh,y,yl):
    yp_pz = L*yh*(1-yh) * p*(1-p)
    return (np.dot(yp_pz, X) - 2*np.dot(y*yp_pz, X)) / yl

def cost_function_pre(syh,syhy):
    loss = syhy/syh  # precision
    return 1-loss

def gradient_descent_pre(p,X,yh,y,syh,syhy):
    yp_pz = L*yh*(1-yh) * p*(1-p)
    return (-np.dot(y*yp_pz, X)*syh + np.dot(yp_pz, X)*syhy) / (syh**2)

def cost_function_rec(sy,syhy):
    loss = syhy/sy  # recall
    return 1-loss

def gradient_descent_rec(p,X,yh,y,sy):
    yp_pz = L*yh*(1-yh) * p*(1-p)
    return -np.dot(y*yp_pz, X) / sy

def cost_function_gmean(sy,syh,syhy,yl):
    loss = (syhy*(yl-syh-sy+syhy)/(sy*(yl-sy)))**0.5        # gmean
    return 1-loss

def gradient_descent_gmean(p,X,yh,y,sy,syh,syhy,yl):
    yp_pz = L*yh*(1-yh) * p*(1-p)
    repeat1 = np.dot(y*yp_pz, X)
    repeat2 = yl-syh-sy+syhy
    return -2*( (repeat1*repeat2) + (-np.dot(yp_pz, X)+repeat1)*syhy ) / (sy*(yl-sy)*syhy*(repeat2))**0.5
def cost_function_balacc(sy,syh,syhy,yl):
    loss = (yl*(syhy+sy)-sy*(syh+sy)) / (2*sy*(yl-sy))     # balanced accuracy
    return 1-loss

def gradient_descent_balacc(p,X,yh,y,sy,yl):
    yp_pz = L*yh*(1-yh) * p*(1-p)
    return -(yl*np.dot(y*yp_pz, X)-sy*np.dot(yp_pz, X))/(2*sy*(yl-sy))

def update_loss(theta,learning_rate,gradient):
    return theta-(learning_rate*gradient)

def plot_cost_function(cost):
    plt.plot(cost,label="loss")
    plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
    plt.ylabel('Loss',fontweight="bold",fontsize = 15)
    plt.title("Cost Function",fontweight="bold",fontsize = 20)
    plt.legend()
    plt.show()

def get_results(y, predicted):
    TN = metrics.confusion_matrix(y, predicted)[0,0]
    FP = metrics.confusion_matrix(y, predicted)[0,1]
    FN = metrics.confusion_matrix(y, predicted)[1,0]
    TP = metrics.confusion_matrix(y, predicted)[1,1]
    acc = (TP+TN)/(TP+TN+FP+FN)

    f1 = TP/(TP + 0.5*(FP+FN))
    f05 = TP/(TP + 0.8*FP + 0.2*FN)
    f2 = TP/(TP + 0.2*FP + 0.8*FN)
    gmean = ((TP/(TP+FN)) * (TN/(TN+FP)))**0.5
    bacc = 0.5*(TP/(TP+FN) + TN/(TN+FP))

    return acc, f1, f05, f2, gmean, bacc

In [7]:
data = []
sample = []
feature = []
imbalance1 = []
imbalance2 = []

for i in range(1, 16):
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Major-project-8th-sem/data_num/ds'+ str(i) +'.csv')
    print('+'*35, '{}th Dataset'.format(i), '+'*35)
    print('<Original Class>\n', df.iloc[:,-1].value_counts())

    MAJOR = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() == max(df.iloc[:,-1].value_counts())].index[0]
    minor = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() != max(df.iloc[:,-1].value_counts())].index[0]
    df.iloc[:,-1] = df.iloc[:,-1].replace(MAJOR, -100)
    df.iloc[:,-1] = df.iloc[:,-1].replace(minor, 1)
    df.iloc[:,-1] = df.iloc[:,-1].replace(-100, 0)
    print('<Modified Class>\n', df.iloc[:,-1].value_counts())
    print('<Imabalance ratio>\n', "{: .2f}:1".format(df.iloc[:,-1].value_counts()[0]/df.iloc[:,-1].value_counts()[1]))
    data.append(i)
    imbalance1.append(df.iloc[:,-1].value_counts()[0]/len(df))
    imbalance2.append(df.iloc[:,-1].value_counts()[1]/len(df))

    X = df.iloc[:, :-1]
    X = (X - X.mean())/X.std()
    y = df.iloc[:, -1]
    sample.append(X.shape[0])
    feature.append(X.shape[1])

+++++++++++++++++++++++++++++++++++ 1th Dataset +++++++++++++++++++++++++++++++++++
<Original Class>
 pathology_cp_data
2    160
1     90
Name: count, dtype: int64
<Modified Class>
 pathology_cp_data
0    160
1     90
Name: count, dtype: int64
<Imabalance ratio>
  1.78:1
+++++++++++++++++++++++++++++++++++ 2th Dataset +++++++++++++++++++++++++++++++++++
<Original Class>
 surgical_lesion
2    154
1     96
Name: count, dtype: int64
<Modified Class>
 surgical_lesion
0    154
1     96
Name: count, dtype: int64
<Imabalance ratio>
  1.60:1
+++++++++++++++++++++++++++++++++++ 3th Dataset +++++++++++++++++++++++++++++++++++
<Original Class>
 Survival_status
1    225
2     81
Name: count, dtype: int64
<Modified Class>
 Survival_status
0    225
1     81
Name: count, dtype: int64
<Imabalance ratio>
  2.78:1
+++++++++++++++++++++++++++++++++++ 4th Dataset +++++++++++++++++++++++++++++++++++
<Original Class>
 Class
1    210
2    100
Name: count, dtype: int64
<Modified Class>
 Class
0    210
1    10

In [8]:
print(data)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [9]:
print(sample)


[250, 250, 306, 310, 320, 327, 328, 335, 336, 349, 351, 358, 363, 365, 381]


In [10]:
print(feature)


[12, 9, 3, 6, 6, 37, 32, 3, 14, 31, 33, 31, 8, 5, 38]


In [11]:
print(imbalance1)


[0.64, 0.616, 0.7352941176470589, 0.6774193548387096, 0.665625, 0.8715596330275229, 0.6890243902439024, 0.8537313432835821, 0.7559523809523809, 0.6217765042979942, 0.6410256410256411, 0.6899441340782123, 0.7658402203856749, 0.915068493150685, 0.847769028871391]


In [12]:
print(imbalance2)


[0.36, 0.384, 0.2647058823529412, 0.3225806451612903, 0.334375, 0.12844036697247707, 0.31097560975609756, 0.14626865671641792, 0.24404761904761904, 0.37822349570200575, 0.358974358974359, 0.3100558659217877, 0.23415977961432508, 0.08493150684931507, 0.15223097112860892]


In [14]:
pd.options.display.max_rows = 20
dummy = list("&"*102)
df_im = pd.DataFrame(data, columns=["data#"] )
df_im["sample"] = sample
df_im["feature"] = feature
df_im["negative(0)"] = imbalance1
df_im["positive(1)"] = imbalance2
df_im

,data#,sample,feature,negative(0),positive(1)
0,1,250,12,0.640000,0.360000
1,2,250,9,0.616000,0.384000
2,3,306,3,0.735294,0.264706
3,4,310,6,0.677419,0.322581
4,5,320,6,0.665625,0.334375
5,6,327,37,0.871560,0.128440
6,7,328,32,0.689024,0.310976
7,8,335,3,0.853731,0.146269
8,9,336,14,0.755952,0.244048
9,10,349,31,0.621777,0.378223


In [15]:
df_im.describe()


,data#,sample,feature,negative(0),positive(1)
count,15.000000,15.000000,15.000000,15.000000,15.000000
mean,8.000000,328.600000,17.866667,0.732402,0.267598
std,4.472136,38.192744,13.783358,0.098664,0.098664
min,1.000000,250.000000,3.000000,0.616000,0.084932
25%,4.500000,315.000000,6.000000,0.653325,0.193195
50%,8.000000,335.000000,12.000000,0.689944,0.310056
75%,11.500000,354.500000,31.500000,0.806805,0.346675
max,15.000000,381.000000,38.000000,0.915068,0.384000


In [16]:
df_im[(df_im["negative(0)"] > 0.7) & (df_im["negative(0)"] <= 0.9)]


,data#,sample,feature,negative(0),positive(1)
2,3,306,3,0.735294,0.264706
5,6,327,37,0.871560,0.128440
7,8,335,3,0.853731,0.146269
8,9,336,14,0.755952,0.244048
12,13,363,8,0.765840,0.234160
14,15,381,38,0.847769,0.152231


**CONDUCTING EXPERIMENTS**

In [17]:
res = pd.DataFrame({'Dataset':['D', 'S', 0, 'C', 'S', 'V'],
                    'Delta':[0, 0, 0, 0, 0, 0],
                    'MSE':[0, 0, 0, 0, 0, 0],
                    'BCE':[0, 0, 0, 0, 0, 0],
                    'AccAim':[0, 0, 0, 0, 0, 0],
                    'F1Aim':[0, 0, 0, 0, 0, 0],
                    'F.5Aim':[0, 0, 0, 0, 0, 0],
                    'F2Aim':[0, 0, 0, 0, 0, 0],
                    'GAim':[0, 0, 0, 0, 0, 0],
                    'BAim':[0, 0, 0, 0, 0, 0]}, index = ['Acc','F1','F.5','F2','Gmean','B_Acc'])

for i in range(1, 16):
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Major-project-8th-sem/data_num/ds'+ str(i) +'.csv')
    print('+'*35, '{}th Dataset'.format(i), '+'*35)
    print('<Original Class>\n', df.iloc[:,-1].value_counts())

    MAJOR = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() == max(df.iloc[:,-1].value_counts())].index[0]
    minor = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() != max(df.iloc[:,-1].value_counts())].index[0]
    df.iloc[:,-1] = df.iloc[:,-1].replace(MAJOR, -100)
    df.iloc[:,-1] = df.iloc[:,-1].replace(minor, 1)
    df.iloc[:,-1] = df.iloc[:,-1].replace(-100, 0)
    print('<Modified Class>\n', df.iloc[:,-1].value_counts())
    print('<Imabalance ratio>\n', "{: .2f}:1".format(df.iloc[:,-1].value_counts()[0]/df.iloc[:,-1].value_counts()[1]))

    X = df.iloc[:, :-1]
    X = (X - X.mean())/X.std()
    y = df.iloc[:, -1]

    ada_acc = []
    ada_f1 = []
    ada_f5 = []
    ada_f2 = []
    ada_gmean = []
    ada_bacc = []

    mse_acc = []
    mse_f1 = []
    mse_f5 = []
    mse_f2 = []
    mse_gmean = []
    mse_bacc = []

    bce_acc = []
    bce_f1 = []
    bce_f5 = []
    bce_f2 = []
    bce_gmean = []
    bce_bacc = []

    acc_acc = []
    acc_f1 = []
    acc_f5 = []
    acc_f2 = []
    acc_gmean = []
    acc_bacc = []

    f1_acc = []
    f1_f1 = []
    f1_f5 = []
    f1_f2 = []
    f1_gmean = []
    f1_bacc = []

    f05_acc = []
    f05_f1 = []
    f05_f5 = []
    f05_f2 = []
    f05_gmean = []
    f05_bacc = []

    f2_acc = []
    f2_f1 = []
    f2_f5 = []
    f2_f2 = []
    f2_gmean = []
    f2_bacc = []

    gmean_acc = []
    gmean_f1 = []
    gmean_f5 = []
    gmean_f2 = []
    gmean_gmean = []
    gmean_bacc = []

    bacc_acc = []
    bacc_f1 = []
    bacc_f5 = []
    bacc_f2 = []
    bacc_gmean = []
    bacc_bacc = []

    from sklearn.model_selection import StratifiedKFold
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state = 2)
    n_iter=0
    for train_index, test_index in skf.split(df, df.iloc[:,-1]):
        n_iter += 1
        X_train = X.iloc[train_index]
        y_train= y.iloc[train_index]
        X_test = X.iloc[test_index]
        y_test= y.iloc[test_index]
        print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        X_test = np.array(X_test)
        y_test = np.array(y_test)

        y_train_p = [-1 if x==0 else x for x in y_train]
        y_test_p = [-1 if x==0 else x for x in y_test]

        intercept_r = np.ones((X_train.shape[0],1))
        intercept_t = np.ones((X_test.shape[0],1))
        X_train_a = np.concatenate((intercept_r,X_train), axis= 1)
        X_test_a = np.concatenate((intercept_t,X_test), axis= 1)

        print('<Class Distribution>\n',
              'Train 0:1\n'.format(i), list(y_train).count(0), list(y_train).count(1), '\n',
              'Test 0:1\n'.format(i), list(y_test).count(0), list(y_test).count(1))


        theta = np.zeros(X_train_a.shape[1])
        num_iter = 1000
        cost = []
        lr = 0.00001

        for j in range(num_iter):
            z = linear(X_train_a,theta)
            cost.append(cost_ada(z,y_train_p))
            gradient = gd_ada(X_train_a,z,y_train_p)
            theta = update_loss(theta,lr,gradient)

        print("------------"*2, "Delta Rule", "------------"*2)
        predicted = predict_p(X_test_a,theta)
        acc, f1, f05, f2, gmean, bacc = get_results(y_test_p, predicted)
        ada_acc.append(acc)
        ada_f1.append(f1)
        ada_f5.append(f05)
        ada_f2.append(f2)
        ada_gmean.append(gmean)
        ada_bacc.append(bacc)

        theta = np.zeros(X_train_a.shape[1])
        num_iter = 1000
        cost = []
        lr = 0.01

        for j in range(num_iter):
            z = linear(X_train_a,theta)
            cost.append(cost_ada2(z,y_train_p))
            gradient = gd_ada2(X_train_a,z,y_train_p)
            theta = update_loss(theta,lr,gradient)

        print("------------"*2, "Delta Rule", "------------"*2)
        predicted = predict_p(X_test_a,theta)
        acc, f1, f05, f2, gmean, bacc = get_results(y_test_p, predicted)
        mse_acc.append(acc)
        mse_f1.append(f1)
        mse_f5.append(f05)
        mse_f2.append(f2)
        mse_gmean.append(gmean)
        mse_bacc.append(bacc)

        threshold = 0.5
        # Logistic Regression (Sigmoid and BCE)
        theta = np.zeros(X_train_a.shape[1])
        num_iter = 1000
        cost = []
        lr = 0.1
        yl = y_train.shape[0]

        for j in range(num_iter):
            h = sigmoid(X_train_a,theta)
            cost.append(cost_function(h,y_train))
            gradient = gradient_descent(X_train_a,h,y_train,yl)
            theta = update_loss(theta,lr,gradient)

        print("------------"*2, "Logistic Regression", "------------"*2)

        predicted = predict(X_test_a,theta)
        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        bce_acc.append(acc)
        bce_f1.append(f1)
        bce_f5.append(f05)
        bce_f2.append(f2)
        bce_gmean.append(gmean)
        bce_bacc.append(bacc)


        # AnyLoss (Accuracy)
        theta = np.zeros(X_train_a.shape[1])
        num_iter = 1000
        L = 73
        cost = []
        lr = 0.005

        sy = np.sum(y_train)
        yl = y_train.shape[0]

        for j in range(num_iter):
            p = sigmoid(X_train_a,theta)
            yh = sigmoid_2(p)
            syh = np.sum(yh)
            syhy = np.dot(yh,y_train)
            cost.append(cost_function_acc(sy,syh,syhy,yl))
            gradient = gradient_descent_acc(p,X_train_a,yh,y_train,yl)
            theta = update_loss(theta,lr,gradient)

        # Train Results
        print("------------"*2, "AnyLoss (Accuracy)", "------------"*2)
        predicted = predict_ours(X_test_a,theta)
        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        acc_acc.append(acc)
        acc_f1.append(f1)
        acc_f5.append(f05)
        acc_f2.append(f2)
        acc_gmean.append(gmean)
        acc_bacc.append(bacc)


        theta = np.zeros(X_train_a.shape[1])
        num_iter = 1000
        beta = 1
        L = 73
        cost = []
        lr = 0.01

        bs = beta**2
        sy = np.sum(y_train)
        bs_sy = bs*sy

        for j in range(num_iter):
            p = sigmoid(X_train_a,theta)
            yh = sigmoid_2(p)
            syh = np.sum(yh)
            syhy = np.dot(yh,y_train)
            cost.append(cost_function_fscore(bs,syh,syhy,bs_sy))
            gradient = gradient_descent_fscore(p,X_train_a,yh,y_train,bs,syh,syhy,bs_sy)
            theta = update_loss(theta,lr,gradient)

        print("------------"*2, "AnyLoss (F1)", "------------"*2)

        predicted = predict_ours(X_test_a,theta)
        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        f1_acc.append(acc)
        f1_f1.append(f1)
        f1_f5.append(f05)
        f1_f2.append(f2)
        f1_gmean.append(gmean)
        f1_bacc.append(bacc)


        # Our Method (F0.5)
        theta = np.zeros(X_train_a.shape[1])
        num_iter = 1000
        beta = 0.5
        L = 73
        cost = []
        lr = 0.005

        bs = beta**2
        sy = np.sum(y_train)
        bs_sy = bs*sy

        for j in range(num_iter):
            p = sigmoid(X_train_a,theta)
            yh = sigmoid_2(p)
            syh = np.sum(yh)
            syhy = np.dot(yh,y_train)
            cost.append(cost_function_fscore(bs,syh,syhy,bs_sy))
            gradient = gradient_descent_fscore(p,X_train_a,yh,y_train,bs,syh,syhy,bs_sy)
            theta = update_loss(theta,lr,gradient)

        # Train Results
        print("------------"*2, "AnyLoss (F0.5)", "------------"*2)

        predicted = predict_ours(X_test_a,theta)
        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        f05_acc.append(acc)
        f05_f1.append(f1)
        f05_f5.append(f05)
        f05_f2.append(f2)
        f05_gmean.append(gmean)
        f05_bacc.append(bacc)



        theta = np.zeros(X_train_a.shape[1])
        num_iter = 1000
        beta = 2
        L = 73
        cost = []
        lr = 0.005

        bs = beta**2
        sy = np.sum(y_train)
        bs_sy = bs*sy

        for j in range(num_iter):
            p = sigmoid(X_train_a,theta)
            yh = sigmoid_2(p)
            syh = np.sum(yh)
            syhy = np.dot(yh,y_train)
            cost.append(cost_function_fscore(bs,syh,syhy,bs_sy))
            gradient = gradient_descent_fscore(p,X_train_a,yh,y_train,bs,syh,syhy,bs_sy)
            theta = update_loss(theta,lr,gradient)

        # Train Results
        print("------------"*2, "AnyLoss (F2)", "------------"*2)
        predicted = predict_ours(X_test_a,theta)
        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        f2_acc.append(acc)
        f2_f1.append(f1)
        f2_f5.append(f05)
        f2_f2.append(f2)
        f2_gmean.append(gmean)
        f2_bacc.append(bacc)


        # AnyLoss (Gmean)
        theta = np.zeros(X_train_a.shape[1])
        num_iter = 1000
        L = 73
        cost = []
        lr = 0.005

        sy = np.sum(y_train)
        yl = y_train.shape[0]

        for j in range(num_iter):
            p = sigmoid(X_train_a,theta)
            yh = sigmoid_2(p)
            syh = np.sum(yh)
            syhy = np.dot(yh,y_train)
            cost.append(cost_function_gmean(sy,syh,syhy,yl))
            gradient = gradient_descent_gmean(p,X_train_a,yh,y_train,sy,syh,syhy,yl)
            theta = update_loss(theta,lr,gradient)

        # Train Results
        print("------------"*2, "AnyLoss (Gmean)", "------------"*2)

        predicted = predict_ours(X_test_a,theta)
        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        gmean_acc.append(acc)
        gmean_f1.append(f1)
        gmean_f5.append(f05)
        gmean_f2.append(f2)
        gmean_gmean.append(gmean)
        gmean_bacc.append(bacc)


        # AnyLoss (B_Accuracy)
        theta = np.zeros(X_train_a.shape[1])
        num_iter = 1000
        L = 73
        cost = []
        lr = 0.005

        sy = np.sum(y_train)
        yl = y_train.shape[0]

        for j in range(num_iter):
            p = sigmoid(X_train_a,theta)
            yh = sigmoid_2(p)
            syh = np.sum(yh)
            syhy = np.dot(yh,y_train)
            cost.append(cost_function_balacc(sy,syh,syhy,yl))
            gradient = gradient_descent_balacc(p,X_train_a,yh,y_train,sy,yl)
            theta = update_loss(theta,lr,gradient)

        # Train Results
        print("------------"*2, "AnyLoss (B_Accuracy)", "------------"*2)
        predicted = predict_ours(X_test_a,theta)
        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        bacc_acc.append(acc)
        bacc_f1.append(f1)
        bacc_f5.append(f05)
        bacc_f2.append(f2)
        bacc_gmean.append(gmean)
        bacc_bacc.append(bacc)


    res.iloc[2,0] = i

    final_ada_acc = np.mean(ada_acc)
    res.iloc[0,1] = final_ada_acc
    final_ada_f1 = np.mean(ada_f1)
    res.iloc[1,1] = final_ada_f1
    final_ada_f5 = np.mean(ada_f5)
    res.iloc[2,1] = final_ada_f5
    final_ada_f2 = np.mean(ada_f2)
    res.iloc[3,1] = final_ada_f2
    final_ada_gmean = np.mean(ada_gmean)
    res.iloc[4,1] = final_ada_gmean
    final_ada_bacc = np.mean(ada_bacc)
    res.iloc[5,1] = final_ada_bacc

    final_mse_acc = np.mean(mse_acc)
    res.iloc[0,2] = final_mse_acc
    final_mse_f1 = np.mean(mse_f1)
    res.iloc[1,2] = final_mse_f1
    final_mse_f5 = np.mean(mse_f5)
    res.iloc[2,2] = final_mse_f5
    final_mse_f2 = np.mean(mse_f2)
    res.iloc[3,2] = final_mse_f2
    final_mse_gmean = np.mean(mse_gmean)
    res.iloc[4,2] = final_mse_gmean
    final_mse_bacc = np.mean(mse_bacc)
    res.iloc[5,2] = final_mse_bacc

    final_bce_acc = np.mean(bce_acc)
    res.iloc[0,3] = final_bce_acc
    final_bce_f1 = np.mean(bce_f1)
    res.iloc[1,3] = final_bce_f1
    final_bce_f5 = np.mean(bce_f5)
    res.iloc[2,3] = final_bce_f5
    final_bce_f2 = np.mean(bce_f2)
    res.iloc[3,3] = final_bce_f2
    final_bce_gmean = np.mean(bce_gmean)
    res.iloc[4,3] = final_bce_gmean
    final_bce_bacc = np.mean(bce_bacc)
    res.iloc[5,3] = final_bce_bacc

    final_acc_acc = np.mean(acc_acc)
    res.iloc[0,4] = final_acc_acc
    final_acc_f1 = np.mean(acc_f1)
    res.iloc[1,4] = final_acc_f1
    final_acc_f5 = np.mean(acc_f5)
    res.iloc[2,4] = final_acc_f5
    final_acc_f2 = np.mean(acc_f2)
    res.iloc[3,4] = final_acc_f2
    final_acc_gmean = np.mean(acc_gmean)
    res.iloc[4,4] = final_acc_gmean
    final_acc_bacc = np.mean(acc_bacc)
    res.iloc[5,4] = final_acc_bacc

    final_f1_acc = np.mean(f1_acc)
    res.iloc[0,5] = final_f1_acc
    final_f1_f1 = np.mean(f1_f1)
    res.iloc[1,5] = final_f1_f1
    final_f1_f5 = np.mean(f1_f5)
    res.iloc[2,5] = final_f1_f5
    final_f1_f2 = np.mean(f1_f2)
    res.iloc[3,5] = final_f1_f2
    final_f1_gmean = np.mean(f1_gmean)
    res.iloc[4,5] = final_f1_gmean
    final_f1_bacc = np.mean(f1_bacc)
    res.iloc[5,5] = final_f1_bacc

    final_f05_acc = np.mean(f05_acc)
    res.iloc[0,6] = final_f05_acc
    final_f05_f1 = np.mean(f05_f1)
    res.iloc[1,6] = final_f05_f1
    final_f05_f5 = np.mean(f05_f5)
    res.iloc[2,6] = final_f05_f5
    final_f05_f2 = np.mean(f05_f2)
    res.iloc[3,6] = final_f05_f2
    final_f05_gmean = np.mean(f05_gmean)
    res.iloc[4,6] = final_f05_gmean
    final_f05_bacc = np.mean(f05_bacc)
    res.iloc[5,6] = final_f05_bacc

    final_f2_acc = np.mean(f2_acc)
    res.iloc[0,7] = final_f2_acc
    final_f2_f1 = np.mean(f2_f1)
    res.iloc[1,7] = final_f2_f1
    final_f2_f5 = np.mean(f2_f5)
    res.iloc[2,7] = final_f2_f5
    final_f2_f2 = np.mean(f2_f2)
    res.iloc[3,7] = final_f2_f2
    final_f2_gmean = np.mean(f2_gmean)
    res.iloc[4,7] = final_f2_gmean
    final_f2_bacc = np.mean(f2_bacc)
    res.iloc[5,7] = final_f2_bacc

    final_gmean_acc = np.mean(gmean_acc)
    res.iloc[0,8] = final_gmean_acc
    final_gmean_f1 = np.mean(gmean_f1)
    res.iloc[1,8] = final_gmean_f1
    final_gmean_f5 = np.mean(gmean_f5)
    res.iloc[2,8] = final_gmean_f5
    final_gmean_f2 = np.mean(gmean_f2)
    res.iloc[3,8] = final_gmean_f2
    final_gmean_gmean = np.mean(gmean_gmean)
    res.iloc[4,8] = final_gmean_gmean
    final_gmean_bacc = np.mean(gmean_bacc)
    res.iloc[5,8] = final_gmean_bacc

    final_bacc_acc = np.mean(bacc_acc)
    res.iloc[0,9] = final_bacc_acc
    final_bacc_f1 = np.mean(bacc_f1)
    res.iloc[1,9] = final_bacc_f1
    final_bacc_f5 = np.mean(bacc_f5)
    res.iloc[2,9] = final_bacc_f5
    final_bacc_f2 = np.mean(bacc_f2)
    res.iloc[3,9] = final_bacc_f2
    final_bacc_gmean = np.mean(bacc_gmean)
    res.iloc[4,9] = final_bacc_gmean
    final_bacc_bacc = np.mean(bacc_bacc)
    res.iloc[5,9] = final_bacc_bacc


    res.to_csv("10CV_SLP_15datasets.csv", mode = 'a', float_format='%.4g')

+++++++++++++++++++++++++++++++++++ 1th Dataset +++++++++++++++++++++++++++++++++++
<Original Class>
 pathology_cp_data
2    160
1     90
Name: count, dtype: int64
<Modified Class>
 pathology_cp_data
0    160
1     90
Name: count, dtype: int64
<Imabalance ratio>
  1.78:1
################################################## 1th CV ##################################################
<Class Distribution>
 Train 0:1
 144 81 
 Test 0:1
 16 9
------------------------ Delta Rule ------------------------
------------------------ Delta Rule ------------------------
------------------------ Logistic Regression ------------------------
------------------------ AnyLoss (Accuracy) ------------------------
------------------------ AnyLoss (F1) ------------------------
------------------------ AnyLoss (F0.5) ------------------------
------------------------ AnyLoss (F2) ------------------------
------------------------ AnyLoss (Gmean) ------------------------
------------------------ AnyLoss (B_Accuracy

<ipython-input-17-db50d140397e>:384: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6559999999999999' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  res.iloc[0,1] = final_ada_acc
<ipython-input-17-db50d140397e>:397: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6439999999999999' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  res.iloc[0,2] = final_mse_acc
<ipython-input-17-db50d140397e>:410: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6599999999999999' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  res.iloc[0,3] = final_bce_acc
<ipython-input-17-db50d140397e>:423: FutureWarning: Setting an item of incompatible d

------------------------ Delta Rule ------------------------
------------------------ Logistic Regression ------------------------
------------------------ AnyLoss (Accuracy) ------------------------
------------------------ AnyLoss (F1) ------------------------
------------------------ AnyLoss (F0.5) ------------------------
------------------------ AnyLoss (F2) ------------------------
------------------------ AnyLoss (Gmean) ------------------------
------------------------ AnyLoss (B_Accuracy) ------------------------
################################################## 2th CV ##################################################
<Class Distribution>
 Train 0:1
 139 86 
 Test 0:1
 15 10
------------------------ Delta Rule ------------------------
------------------------ Delta Rule ------------------------
------------------------ Logistic Regression ------------------------
------------------------ AnyLoss (Accuracy) ------------------------
------------------------ AnyLoss (F1) ------

<ipython-input-6-807460e33b18>:37: RuntimeWarning: divide by zero encountered in log
  loss = ((-y * np.log(h))-((1-y)* np.log(1-h))).mean()
<ipython-input-6-807460e33b18>:37: RuntimeWarning: invalid value encountered in multiply
  loss = ((-y * np.log(h))-((1-y)* np.log(1-h))).mean()


------------------------ AnyLoss (Accuracy) ------------------------
------------------------ AnyLoss (F1) ------------------------
------------------------ AnyLoss (F0.5) ------------------------
------------------------ AnyLoss (F2) ------------------------
------------------------ AnyLoss (Gmean) ------------------------
------------------------ AnyLoss (B_Accuracy) ------------------------
################################################## 3th CV ##################################################
<Class Distribution>
 Train 0:1
 290 53 
 Test 0:1
 33 5
------------------------ Delta Rule ------------------------
------------------------ Delta Rule ------------------------


...

------------------------ AnyLoss (Accuracy) ------------------------
------------------------ AnyLoss (F1) ------------------------
------------------------ AnyLoss (F0.5) ------------------------
------------------------ AnyLoss (F2) ------------------------
------------------------ AnyLoss (Gmean) ------------------------
------------------------ AnyLoss (B_Accuracy) ------------------------
################################################## 9th CV ##################################################
<Class Distribution>
 Train 0:1
 291 52 
 Test 0:1
 32 6
------------------------ Delta Rule ------------------------
------------------------ Delta Rule ------------------------


<ipython-input-6-807460e33b18>:37: RuntimeWarning: divide by zero encountered in log
  loss = ((-y * np.log(h))-((1-y)* np.log(1-h))).mean()
<ipython-input-6-807460e33b18>:37: RuntimeWarning: invalid value encountered in multiply
  loss = ((-y * np.log(h))-((1-y)* np.log(1-h))).mean()


------------------------ Logistic Regression ------------------------
------------------------ AnyLoss (Accuracy) ------------------------
------------------------ AnyLoss (F1) ------------------------
------------------------ AnyLoss (F0.5) ------------------------
------------------------ AnyLoss (F2) ------------------------
------------------------ AnyLoss (Gmean) ------------------------
------------------------ AnyLoss (B_Accuracy) ------------------------
################################################## 10th CV ##################################################
<Class Distribution>
 Train 0:1
 291 52 
 Test 0:1
 32 6
------------------------ Delta Rule ------------------------
------------------------ Delta Rule ------------------------
------------------------ Logistic Regression ------------------------


<ipython-input-6-807460e33b18>:37: RuntimeWarning: divide by zero encountered in log
  loss = ((-y * np.log(h))-((1-y)* np.log(1-h))).mean()
<ipython-input-6-807460e33b18>:37: RuntimeWarning: invalid value encountered in multiply
  loss = ((-y * np.log(h))-((1-y)* np.log(1-h))).mean()


------------------------ AnyLoss (Accuracy) ------------------------
------------------------ AnyLoss (F1) ------------------------
------------------------ AnyLoss (F0.5) ------------------------
------------------------ AnyLoss (F2) ------------------------
------------------------ AnyLoss (Gmean) ------------------------
------------------------ AnyLoss (B_Accuracy) ------------------------


In [18]:
res

,Dataset,Delta,MSE,BCE,AccAim,F1Aim,F.5Aim,F2Aim,GAim,BAim
Acc,D,0.868758,0.866127,0.873887,0.852969,0.860864,0.858165,0.750810,0.816059,0.790081
F1,S,0.250794,0.250794,0.384266,0.328889,0.441469,0.375404,0.377699,0.438016,0.391710
F.5,15,0.359524,0.359524,0.451064,0.395455,0.473213,0.427935,0.321238,0.388801,0.341338
F2,C,0.194807,0.194807,0.343169,0.289153,0.429026,0.338848,0.465726,0.503073,0.465772
Gmean,S,0.314696,0.314696,0.461380,0.418076,0.574132,0.481898,0.604797,0.611281,0.582812
B_Acc,V,0.581922,0.580360,0.647699,0.613172,0.682367,0.636705,0.671619,0.710161,0.686288
